# Simulated Agent

In [15]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import binom, beta

In [16]:
class Agent:
    def __init__(self, a0, b0, q):
        self.a0 = a0
        self.b0 = b0
        self.q = q
        self.p = 0.0
        self.pickP()
        #q = p(x|past)
        
    def pickP(self):#PICK P  FROM BETA DISTRIBUTION
        self.p = np.random.beta(self.a0, self.b0)
    # def notPickP(self):
    #     self.p = 1 - np.random.beta(self.a0, self.b0)
            
    def draw(self):
        if np.random.uniform(0,1) < self.p:
            return True
        else:
            return False
        
    def turn(self):
        if np.random.uniform(0,1) < self.q:
            self.pickP()
            
        return self.draw()

In [49]:
# np.random.seed(42)
ag = Agent(0.1,0.5, 0.1)
n = 20
outcomes = []
prob = []
for i in range(n):
    outcomes.append(ag.turn())


In [50]:
outcomes

[True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [77]:
class DBMAgent:
    def __init__(self, a0, b0, gamma):
        self.a0 = a0
        self.b0 = b0
        self.gamma = gamma
        self.p = 0.0
        # self.outcomes = outcomes
        self.num = len(outcomes)
        self.tb = np.zeros((self.num + 1, self.num + 1))
        self.tb[0, 0] = 1
        

    # def mu(self, n, m):
    #     self.a0 = self.a0 + n
    #     self.b0 = self.b0 + m
    #     return self.a0 / (self.a0 + self.b0)
        

    def update(self, outcomes):
        # print(outcomes, '\n',self.tb)
        for o in outcomes:
           
            tb_new = self.tb.copy()
            
            for n in range(self.tb.shape[0]):
                for m in range(self.tb.shape[1]):
                    if n==0 and m==0:
                        self.tb[n, m] = self.tb[0,0] + self.gamma
                    else:
                        self.tb[n, m] = self.tb[n,m] * (1.0 - self.gamma)
                    # print(self.tb[0])

            for n in range(self.tb.shape[0]):
                for m in range(self.tb.shape[1]):
                    #updating each lever
                    if o: 
                        # if n > 0:
                        #     tb_new[n, m] = self.tb[n-1, m] * beta.mean(self.a0 + n, self.b0 + (m-1))
                        if m > 0:
                            tb_new[n, m] = self.tb[n, m-1] * beta.mean(self.a0 + n, self.b0 + (m-1))
                        else:
                            tb_new[n, m] = 0.0
                    else:
                        # if m > 0:
                        #     tb_new[n, m] = self.tb[n, m-1] * self.beta.mean(self.a0 + n, self.b0 +  m)
                        if n > 0:
                            tb_new[n, m] = self.tb[n-1, m] * (1- beta.mean(self.a0 + (n-1), self.b0 + m) )
                        else:
                            tb_new[n, m] = 0.0
            # for n in range(self.tb.shape[0]):
            #     for m in range(self.tb.shape[1]):
            #         if n==0 and m==0:
            #             self.tb[n, m] = self.tb[0,0] + self.gamma
            #         else:
            #             self.tb[n, m] = self.tb[n,m] * (1.0 - self.gamma)
                    # print(self.tb)
            
            z = 0.0
            for n in range(self.tb.shape[0]):
                for m in range(self.tb.shape[1]):
                    z += tb_new[n, m] 
                    
            for n in range(self.tb.shape[0]):
                for m in range(self.tb.shape[1]):
                    self.tb[n, m] = tb_new[n, m] / z    
                           
            # print(o, z,'\n', self.tb[0])


            
agent = DBMAgent(0.1, 0.5, 0.2)
agent.update(outcomes)




In [90]:
agent.tb

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [4.54232535e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [3.05166291e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.0000

In [78]:
class logL:
    def __init__(self, data):
        self.data = data
    def __call__(self, params): # matrix of params
        a0,b0,gm = params 
        mdl = DBMAgent(a0,b0,gm)
        mdl.update(self.data)
        likelihood = 0.0
        # likelihood = gm * beta.pdf(self.data, a0, b0) + (1 - gm) * beta.pdf(self.data, a0, b0)
        # loglikelihood = -1 * np.sum(np.log(likelihood))
        for i in range(len(self.data)):
            for n in range(mdl.tb.shape[0]):
                for m in range(mdl.tb.shape[1]):
                    reward = self.data[i]
                    # prob = beta.mean(model.alpha, model.beta)
                    likelihood += (gm * mdl.tb[n,m] + (1 - gm) * (1 if n==0 and m==0 else 0))
                    log_likelihood = np.log(likelihood)
                    # model.update(reward)
        
        return -log_likelihood
    

In [79]:
logl = logL(outcomes)
p = np.array([0.1,0.5,0.2])
logl(p)


-2.9957322735539917

In [80]:
optimised_result = minimize(logl,p, method='Nelder-Mead')
alpha_mle, beta_mle, gamma_mle = optimised_result.x

In [81]:
print(f'optimised alpha: {alpha_mle}\noptimised beta: {beta_mle}\noptimised gamma: {gamma_mle}')

optimised alpha: 0.09750000000000002
optimised beta: 0.5125
optimised gamma: 0.20500000000000004


In [82]:
optimised_result

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -2.995732273553993
             x: [ 9.750e-02  5.125e-01  2.050e-01]
           nit: 18
          nfev: 54
 final_simplex: (array([[ 9.750e-02,  5.125e-01,  2.050e-01],
                       [ 9.750e-02,  5.126e-01,  2.050e-01],
                       [ 9.750e-02,  5.125e-01,  2.050e-01],
                       [ 9.749e-02,  5.125e-01,  2.050e-01]]), array([-2.996e+00, -2.996e+00, -2.996e+00, -2.996e+00]))

In [ ]:
            
#         for o in self.outcomes:

#             for n in range(self.tb.shape[0]):
#                 for m in range(self.tb.shape[1]):
#                     if n==0 and m==0:
#                         self.tb[n, m] = self.tb[0,0]+self.gamma
#                     else:
#                         self.tb[n, m] = self.tb[n,m]*(1.0 - self.gamma)

            
#             tb_new=self.tb.copy()
            
#             for n in range(self.tb.shape[0]):
#                 for m in range(self.tb.shape[1]):
                        
#                     if o:
#                         if m>0:
#                             tb_new[n,m]=self.tb[n,m-1]*beta.mean(self.a0 + n, self.b0+m)
#                         else:
#                             tb_new[n,m]=0.0
                            
#                     else:
#                         if n>0:
#                             tb_new[n,m]=self.tb[n-1,m]*beta.mean(self.a0 + n, self.b0+m)
#                         else:
#                             tb_new[n,m]=0.0

#             z=0.0                    
#             for n in range(self.tb.shape[0]):
#                 for m in range(self.tb.shape[1]):
#                     z+=tb_new[n,m]

#             for n in range(self.tb.shape[0]):
#                 for m in range(self.tb.shape[1]):
#                     self.tb[n,m]=tb_new[n,m]/z
                
#             print(o,'\n', self.tb)